## 【コロモゴロフスミルノフ検定（KS検定）】

In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
import numpy as np
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features, reduce_mem_usage

from tqdm import tqdm
import time
import sys

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

In [ ]:
import glob
#  Data Load
base = utils.read_df_pkl('../input/base_term*0*')
win_path = f'../features/4_winner/*.gz'
win_path = f'../features/3_third_valid/*.gz'
win_path_list = glob.glob(win_path)
tmp_path_list = glob.glob('../features/5_tmp/*.gz')
# tmp_path_list = []
win_path_list += tmp_path_list

path_list = win_path_list

base_train = base[~base[target].isnull()].reset_index(drop=True)
base_test = base[base[target].isnull()].reset_index(drop=True)
feature_list = utils.parallel_load_data(path_list=path_list)
df = pd.concat(feature_list, axis=1)
train = pd.concat([base_train, df.iloc[:len(base_train), :]], axis=1)
test = pd.concat([base_test, df.iloc[len(base_train):, :].reset_index(drop=True)], axis=1)

100%|██████████| 3/3 [00:00<00:00, 18.08it/s]


In [ ]:
from scipy.stats import ks_2samp
from joblib import Parallel, delayed
list_p_value =[]

for col in tqdm(df.columns):
    list_p_value.append(ks_2samp(test[col] , train[col])[1])

se_p = pd.Series(list_p_value, index = df.columns).sort_values() 
list_discarded = list(se_p[se_p < .1].index)
list_discarded = list(set( list_discarded))

In [13]:
import shutil
for path in path_list:
    for col in list_discarded:
        if path.count(col):
            try:
                shutil.move(path, '../features/2_second_valid/')
            except FileNotFoundError:
                pass

In [13]:
import shutil
import re
import os

feature_season = 'season1_features'

list_discarded = list(set( list_discarded))
for feat in list_discarded:
    if feat.count('target'):continue
    train_path = f'{feat[:8]}train_{feat[8:]}.gz'.replace('2_1_0', '2_1.0').replace('2_3_0', '2_3.0')
    test_path = f'{feat[:8]}test_{feat[8:]}.gz'.replace('2_1_0', '2_1.0').replace('2_3_0', '2_3.0')
    if  os.path.exists(f'../{feature_season}/3_third_valid/{train_path}'):
        continue
    if  os.path.exists(f'../{feature_season}/3_third_valid/{test_path}'):
        continue
    shutil.move(f'../{feature_season}/4_winner/{train_path}', f'../{feature_season}/3_third_valid/')
    shutil.move(f'../{feature_season}/4_winner/{test_path}', f'../{feature_season}/3_third_valid/')